# Covertype Data Set Preprocessing

In [ ]:
%pip install gcsfs

In [1]:
import numpy as np
import pandas as pd
import tensorflow_data_validation as tfdv


from sklearn.model_selection import train_test_split

### Set the paths

In [34]:
FULL_DATASET = '../covertype.csv'
TRAINING_DATASET='../covertype_training.csv'
EVALUATION_DATASET='../covertype_evaluation.csv'
EVALUATION_DATASET_WITH_ANOMALIES='../covertype_evaluation_anomalies.csv'
SERVING_DATASET='../covertype_serving.csv'

ORIGINAL_DATASET_PATH = 'gs://workshop-datasets/covertype/orig/covtype.data'

### Load the dataset

In [3]:
df = pd.read_csv(ORIGINAL_DATASET_PATH, header=None)
print(df.shape)
df.head()

(581012, 55)


,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


### Configure soil type and wilderness area domains

In [7]:
soil_type = [
"1", "2702", "Cathedral family - Rock outcrop complex, extremely stony.",
"2", "2703", "Vanet - Ratake families complex, very stony.",
"3", "2704", "Haploborolis - Rock outcrop complex, rubbly.",
"4", "2705", "Ratake family - Rock outcrop complex, rubbly.",
"5", "2706", "Vanet family - Rock outcrop complex complex, rubbly.",
"6", "2717", "Vanet - Wetmore families - Rock outcrop complex, stony.",
"7", "3501", "Gothic family.",
"8", "3502", "Supervisor - Limber families complex.",
"9", "4201", "Troutville family, very stony.",
"10", "4703", "Bullwark - Catamount families - Rock outcrop complex, rubbly.",
"11", "4704", "Bullwark - Catamount families - Rock land complex, rubbly.",
"12", "4744", "Legault family - Rock land complex, stony.",
"13", "4758", "Catamount family - Rock land - Bullwark family complex, rubbly.",
"14", "5101", "Pachic Argiborolis - Aquolis complex.",
"15", "5151", "unspecified in the USFS Soil and ELU Survey.",
"16", "6101", "Cryaquolis - Cryoborolis complex.",
"17", "6102", "Gateview family - Cryaquolis complex.",
"18", "6731", "Rogert family, very stony.",
"19", "7101", "Typic Cryaquolis - Borohemists complex.",
"20", "7102", "Typic Cryaquepts - Typic Cryaquolls complex.",
"21", "7103", "Typic Cryaquolls - Leighcan family, till substratum complex.",
"22", "7201", "Leighcan family, till substratum, extremely bouldery.",
"23", "7202", "Leighcan family, till substratum - Typic Cryaquolls complex.",
"24", "7700", "Leighcan family, extremely stony.",
"25", "7701", "Leighcan family, warm, extremely stony.",
"26", "7702", "Granile - Catamount families complex, very stony.",
"27", "7709", "Leighcan family, warm - Rock outcrop complex, extremely stony.",
"28", "7710", "Leighcan family - Rock outcrop complex, extremely stony.",
"29", "7745", "Como - Legault families complex, extremely stony.",
"30", "7746", "Como family - Rock land - Legault family complex, extremely stony.",
"31", "7755", "Leighcan - Catamount families complex, extremely stony.",
"32", "7756", "Catamount family - Rock outcrop - Leighcan family complex, extremely stony.",
"33", "7757", "Leighcan - Catamount families - Rock outcrop complex, extremely stony.",
"34", "7790", "Cryorthents - Rock land complex, extremely stony.",
"35", "8703", "Cryumbrepts - Rock outcrop - Cryaquepts complex.",
"36", "8707", "Bross family - Rock land - Cryumbrepts complex, extremely stony.",
"37", "8708", "Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.",
"38", "8771", "Leighcan - Moran families - Cryaquolls complex, extremely stony.",
"39", "8772", "Moran family - Cryorthents - Leighcan family complex, extremely stony.",
"40", "8776", "Moran family - Cryorthents - Rock land complex, extremely stony.",
]

wilderness_area = [
"Rawah", "Rawah Wilderness Area",
"Neota", "Neota Wilderness Area",
"Commanche", "Comanche Peak Wilderness Area",
"Cache", "Cache la Poudre Wilderness Area"
]

In [12]:
wilderness_area[0::2]

['Rawah', 'Neota', 'Commanche', 'Cache']

### Map one-hot encoded values to categorical domains

In [6]:
soil = df.loc[:, 14:53].apply(lambda x: soil_type[1::3][x.to_numpy().nonzero()[0][0]], axis=1)
soil

0         7745
1         7745
2         4744
3         7746
4         7745
          ... 
581007    2703
581008    2703
581009    2703
581010    2703
581011    2703
Length: 581012, dtype: object

In [13]:
wilderness = df.loc[:, 10:13].apply(lambda x: wilderness_area[0::2][x.to_numpy().nonzero()[0][0]], axis=1)
wilderness

0             Rawah
1             Rawah
2             Rawah
3             Rawah
4             Rawah
            ...    
581007    Commanche
581008    Commanche
581009    Commanche
581010    Commanche
581011    Commanche
Length: 581012, dtype: object

### Create a dataset with column names and categorical values replacing one-hot encoded soil type and wilderness areas

In [14]:
COLUMN_NAMES = [
    'Elevation', 
    'Aspect', 
    'Slope', 
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points',
    'Wilderness_Area',
    'Soil_Type',
    'Cover_Type']

df_processed = pd.concat([df.loc[:, 0:9], wilderness, soil, df.loc[:, 54]], axis=1, ignore_index=True)
df_processed.columns = COLUMN_NAMES
df_processed

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,7745,5
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,7745,5
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,4744,2
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,7746,2
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,7745,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,2703,3
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,2703,3
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,2703,3
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,2703,3


### Save the dataset to CSV file

In [15]:
df_processed.to_csv(FULL_DATASET, header=True, index=False)

### Create training, validation, testing and serving splits.

In [27]:
df_train, df_other = train_test_split(df_processed, train_size=431012, shuffle=True)
df_evaluate = df_other.iloc[0:75000]
df_serving = df_other.iloc[75000:150000]
df_serving = df_serving.drop(columns=['Cover_Type'])
print(df_train.shape)
print(df_evaluate.shape)
print(df_serving.shape)

(431012, 13)
(75000, 13)
(75000, 12)


In [28]:
df_train.to_csv(TRAINING_DATASET, header=True, index=False)
df_evaluate.to_csv(EVALUATION_DATASET, header=True, index=False)
df_serving.to_csv(SERVING_DATASET, header=True, index=False)

### Create the evaluation split with anomalies.

Create the evaluation split where some values of Slope are more than 90 degrees and some examples have 2701 code for soil type (2701 is not present in the training split)

In [66]:
df_evaluate.loc[[89916, 410521, 559685], 'Slope'] = 110
df_evaluate.loc[[317232, 401376], 'Soil_Type'] = 2701
df_evaluate

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
89916,3004,49,110,108,14,3551,224,218,127,5309,Rawah,7745,2
410521,3113,24,110,180,37,1194,187,159,92,942,Rawah,7201,1
559685,2531,156,110,30,-9,1571,238,238,119,1443,Commanche,2703,2
317232,2824,155,17,153,50,781,238,239,125,2981,Commanche,2701,2
401376,3133,61,11,258,23,2040,229,218,121,3188,Commanche,2701,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162082,2978,351,5,268,3,2843,212,232,158,2008,Rawah,7745,2
360883,2691,98,12,120,22,2400,239,223,113,680,Commanche,4703,2
239051,2888,45,12,127,1,872,223,213,122,2075,Commanche,7700,1
409871,2996,279,25,484,174,2820,142,234,226,518,Commanche,4758,2


In [67]:
df_evaluate.to_csv(EVALUATION_DATASET_WITH_ANOMALIES, header=True, index=False)